In [ ]:
from main import *
from models import *
from tqdm import tqdm

beta_1 = 1e-4
beta_T = 0.02
T = 500
shape = (44, 188, 4)
device = torch.device('cuda')

total_iteration = 2000

train_sub = [1,2,3,4,5,6,7,8]

train_x0, train_y0 = make_train_dataset(train_sub, [0.0])
train_x1, train_y1 = make_train_dataset(train_sub, [1.0])


print(train_x0[0].shape)
train_stft0 = Stft_dataset(train_x0, train_y0)
train_stft1 = Stft_dataset(train_x1, train_y1)

score_model0 = DiffusionModel(device, beta_1, beta_T, T).to(device)
score_model1 = DiffusionModel(device, beta_1, beta_T, T).to(device)


dataloader0 = torch.utils.data.DataLoader(train_stft0, batch_size=32, shuffle=True, num_workers=0, drop_last=True)
optim0 = torch.optim.Adam(score_model0.parameters(), lr = 0.0001)

dataloader1 = torch.utils.data.DataLoader(train_stft1, batch_size=32, shuffle=True, num_workers=0, drop_last=True)
optim1 = torch.optim.Adam(score_model1.parameters(), lr = 0.0001)


total_iteration = 5000


pbar = tqdm(range(total_iteration))

for epoch in pbar:
    losses = []
    num_items = 0

    for x, y in dataloader0:
        data = x.to(device = device)
        y = y.to(device = device)
        loss = loss_fn(score_model0, data, idx=None, y=y)

        optim0.zero_grad()
        loss.backward()
        optim0.step()

        losses.append(loss.item())
        num_items += x.shape[0]
    pbar.set_description("Average Loss : {}".format(sum(losses) / num_items))
    if ((sum(losses) / num_items) < 8e-5):
        break

for epoch in pbar:
    losses = []
    num_items = 0

    for x, y in dataloader1:
        data = x.to(device = device)
        y = y.to(device = device)
        loss = loss_fn(score_model1, data, idx=None, y=y)

        optim1.zero_grad()
        loss.backward()
        optim1.step()

        losses.append(loss.item())
        num_items += x.shape[0]
    pbar.set_description("Average Loss : {}".format(sum(losses) / num_items))
    if ((sum(losses) / num_items) < 8e-5):
        break


(44, 188, 4)


  0%|                                                                                                                                                                                                                              | 0/5000 [00:00<?, ?it/s]/home/junyeobe/.local/lib/python3.9/site-packages/torch/nn/functional.py:3737: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
Average Loss : 0.0003827872618179147:   3%|█████▉                                                                                                                                                                      | 172/5000 [12:55<6:05:24,  4.54s/it]

In [ ]:
only_final = True
process0 = DiffusionProcess(beta_1, beta_T, T, score_model0, device, shape)
process1 = DiffusionProcess(beta_1, beta_T, T, score_model1, device, shape)

samples0 = sample(process0, label=0, num_images=32, device="cuda")
samples1 = sample(process1, label=1, num_images=32, device="cuda")


In [ ]:
from sklearn.manifold import TSNE
from numpy import reshape
import seaborn as sns
import pandas as pd  

import warnings

warnings.filterwarnings("ignore")


x0 = np.array(samples0).reshape(32,-1)
y0 = np.array([0 for i in range(32)])

x1 = np.array(samples1).reshape(32,-1)
y1 = np.array([1 for i in range(32)])

x = np.concatenate([x0,x1])
y = np.concatenate([y0,y1])

tsne = TSNE(n_components=2, verbose=1)
z = tsne.fit_transform(x) 

df = pd.DataFrame()
df["y"] = y
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

In [ ]:
import matplotlib.pyplot as plt


sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 4),
                data=df).set(title="data T-SNE projection") 
plt.show()

vis = samples0[0][0]
for channel in vis:
    plt.plot(vis)
    plt.show()

In [ ]:
from main import *

train_sub = [1]
val_sub = [2]
test_sub = [2]

device = 'cuda'
main(train_sub, val_sub, test_sub, device)


In [ ]:
# 실험결과 
# 그냥 stft 바로 집어넣었을 때 loss 0.002 부근

# stft / 10 했을 때 loss 더 줄어든다
